In [1]:
using Plots


In [10]:
using LinearAlgebra

In [136]:
module Optimizers
using LinearAlgebra

"""
    QPeq_Schur(Q, Qinv, c, A, b)

Solve an equality-only quadratic programming problem

# Arguments

- `Q::AbstractMatrix{T}`: the matrix Q of the quadratic cost

- `Qinv::AbstractMatrix{T}`: the inverse of matrix Q

- `c::AbstractVector{T}`: the vector c of the quadratic cost

- `A::AbstractMatrix{T}`: the matrix A of the equality constraints

- `b::AbstractVector{T}`: the vector b of the equality constraints

"""
function QPeq_Schur(Q, Qinv, c, A, b)
    lambda_star = (A * Qinv * A') \ (-b + A * Qinv * c)
    xstar = - Qinv * (c + A' * lambda_star)
    
    fxstar = 1/2 * xstar' * Q * xstar + c' * xstar
    
    return xstar, fxstar, lambda_star
end

"""
    QPeq_Schur_autoinv(Q, Qinv, c, A, b)

Solve an equality-only quadratic programming problem, avoiding the inverse
computation of Q

# Arguments

- `Q::AbstractMatrix{T}`: the matrix Q of the quadratic cost

- `c::AbstractVector{T}`: the vector c of the quadratic cost

- `A::AbstractMatrix{T}`: the matrix A of the equality constraints

- `b::AbstractVector{T}`: the vector b of the equality constraints

"""
function QPeq_Schur_autoinv(Q, c, A, b)
    Qinvc = Q \ c
    QinvAt = Q \ A'
    
    lambda_star = (A * QinvAt) \ (-b + A * Qinvc)
    xstar = - Qinvc - QinvAt * lambda_star
    
    fxstar = 1/2 * xstar' * Q * xstar + c' * xstar
    
    return xstar, fxstar, lambda_star
end

function QPeq_Null(Q, c, A, b, x2)
    m, n = size(A)
    A1 = A[1:m, 1:m]
    A2 = A[1:m, (m+1):n]
    
    xhat = [inv(A1)*(b - A2*x2); x2]
    
    Z = [(-inv(A1) * A2); I(n-m)]
    
    v_star = (Z' * Q * Z) \ (- Z' * (c + Q * xhat))
    xstar = Z * v_star + xhat
    lambda_star = (A * A') \ (-A * (c + Q * xstar))
    
    fxstar = 1/2 * xstar' * Q * xstar + c' * xstar
    
    return xstar, fxstar, lambda_star, v_star
end


end

import Main.Optimizers

n = 50
m = 10
A = rand(m, n)
@assert rank(A) == m
b = rand(m)
c = rand(n)
Q = diagm(rand(n))

@assert isapprox(Optimizers.QPeq_Schur(Q, inv(Q), c, A, b)[1], Optimizers.QPeq_Schur_autoinv(Q, c, A, b)[1])
@assert isapprox(Optimizers.QPeq_Schur(Q, inv(Q), c, A, b)[2], Optimizers.QPeq_Schur_autoinv(Q, c, A, b)[2])
@assert isapprox(Optimizers.QPeq_Schur(Q, inv(Q), c, A, b)[3], Optimizers.QPeq_Schur_autoinv(Q, c, A, b)[3])
;

A1 = A[1:m, 1:m]
A2 = A[1:m, (m+1):n]
x2 = [inv(A1)*b; zeros(n - m - m)]

@show Optimizers.QPeq_Schur(Q, inv(Q), c, A, b)[1][1:5]
@show Optimizers.QPeq_Null(Q, c, A, b, x2)[1][1:5]
;

((Optimizers.QPeq_Schur(Q, inv(Q), c, A, b))[1])[1:5] = [-13.318342239632344, -2.355529992263436, -41.03933405928532, -0.8791618885312952, -3.139410164264138]
((Optimizers.QPeq_Null(Q, c, A, b, x2))[1])[1:5] = 

[-0.8276005678536449, -0.12475861709200808, 0.9518763888177866, 0.8028359519672044, 1.115510655564293]
